In [1]:
%matplotlib inline
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.sectorperformance import SectorPerformances
from alpha_vantage.cryptocurrencies import CryptoCurrencies
from pandas import DataFrame, read_csv
from datetime import datetime
from pandas.plotting import register_matplotlib_converters
import csv
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from datetime import date, timedelta

In [2]:
def unifyDate(date):
    if "/" in date:
        dates = date.split("/")
        return dates[2]+"-"+dates[0]+"-"+dates[1];
    else:
        return date;

In [3]:
Location = r'C:\Users\Eazy\Documents\CSE\CSE410\Project\Saved_CSV\MACD.csv'
macd = pd.read_csv(Location)
macd_dates = macd['date'].to_numpy()
macd_hist = macd['MACD_Hist'].to_numpy()
macd_data = macd['MACD'].to_numpy()
macd_signal = macd['MACD_Signal'].to_numpy()

len(macd)

4998

In [4]:
Location = r'C:\Users\Eazy\Documents\CSE\CSE410\Project\Saved_CSV\RSI.csv'
rsi = pd.read_csv(Location)
rsi_dates = rsi['date'].to_numpy()
rsi_data = rsi['RSI'].to_numpy()

rsi_dates[0]

'1999-11-12'

In [5]:
Location = r'C:\Users\Eazy\Documents\CSE\CSE410\Project\Saved_CSV\Price.csv'
price = pd.read_csv(Location)
price_close = price['4. close'].to_numpy()
date_arr = price['date'].to_numpy()
price_close

array([ 77.625,  80.25 ,  81.5  , ..., 243.29 , 243.26 , 248.165])

In [6]:
next_day = price_close[1:]
next_day
#Be aware of next_day is 1 element less than price_close

array([ 80.25 ,  81.5  ,  83.625, ..., 243.29 , 243.26 , 248.165])

In [13]:
#Array structure: Date, Price, SMA_200, SMA_50, WOO rating

sma_200 = r'C:\Users\Eazy\Documents\CSE\CSE410\Project\Saved_CSV\SMA_200.csv'
sma_50 = r'C:\Users\Eazy\Documents\CSE\CSE410\Project\Saved_CSV\SMA_50.csv'

sma_200_data = pd.read_csv(sma_200).to_numpy()
sma_50_data = pd.read_csv(sma_50).to_numpy()
first_sma_200 = sma_200_data[0]
first_sma_50 = sma_50_data[0]

toAdd_200 = first_sma_200[1]/200  #first day SMA / 200 
toAdd_50 = first_sma_50[1]/50     #first day SMA / 50

sma_200 = toAdd_200   #day 1
sma_50 = toAdd_50

l = [None] * len(sma_200_data)

#filling the gap of first 200 days
for i in range(198):
    d = date_arr[i]
    #use real sma_50 data after day 50
    val = 0
    if i >= 50:
        val = sma_50_data[i-49][1]
    else:
        val = sma_50
    
    woo=0
    if price_close[i] > sma_50 and sma_50 > sma_200:
        woo = 1;
    elif  sma_50 > price_close[i] and price_close[i] > sma_200:
        woo = 2;
    elif  sma_50 > sma_200 and sma_200 > price_close[i]:
        woo = 3;
    elif  sma_200 > sma_50 and sma_50 > price_close[i]:
        woo = 4;
    elif  sma_200 > price_close[i] and price_close[i] > sma_50:
        woo = 5;
    elif  price_close[i] > sma_200 and sma_200 > sma_50:
        woo = 6;
    l[i] = [d, price_close[i], sma_200, sma_50, woo]
    sma_200 = sma_200 + toAdd_200
    sma_50 = sma_50 + toAdd_50

#add real sma_200_day data into previous array
for i in range(199,len(sma_200_data)):
    d = date_arr[i]
    woo = 0
    if price_close[i] > sma_50_data[i-49][1] and sma_50_data[i-49][1] > sma_200_data[i-199][1]:
        woo = 1;
    elif  sma_50_data[i-49][1] > price_close[i] and price_close[i] > sma_200_data[i-199][1]:
        woo = 2;
    elif  sma_50_data[i-49][1] > sma_200_data[i-199][1] and sma_200_data[i-199][1] > price_close[i]:
        woo = 3;
    elif  sma_200_data[i-199][1] > sma_50_data[i-49][1] and sma_50_data[i-49][1] > price_close[i]:
        woo = 4;
    elif  sma_200_data[i-199][1] > price_close[i] and price_close[i] > sma_50_data[i-49][1]:
        woo = 5;
    elif  price_close[i] > sma_200_data[i-199][1] and sma_200_data[i-199][1] > sma_50_data[i-49][1]:
        woo = 6;
    l[i] = [d, price_close[i], sma_200_data[i-199][1], sma_50_data[i-49][1], woo]
    
#l: date, closing price, sma_200, sma_50, woo rating
l


[['1999-11-01', 77.625, 0.4827815, 1.931126, 1],
 ['1999-11-02', 80.25, 0.965563, 3.862252, 1],
 ['1999-11-03', 81.5, 1.4483445, 5.793378, 1],
 ['1999-11-04', 83.625, 1.931126, 7.724504, 1],
 ['1999-11-05', 88.31299999999999, 2.4139074999999997, 9.655629999999999, 1],
 ['1999-11-08', 96.375, 2.8966889999999994, 11.586755999999998, 1],
 ['1999-11-09', 89.625, 3.379470499999999, 13.517881999999997, 1],
 ['1999-11-10', 91.43799999999999, 3.862251999999999, 15.449007999999996, 1],
 ['1999-11-11', 92.25, 4.345033499999999, 17.380133999999995, 1],
 ['1999-11-12', 90.625, 4.827814999999998, 19.311259999999994, 1],
 ['1999-11-15', 89.43799999999999, 5.310596499999998, 21.242385999999993, 1],
 ['1999-11-16', 91.18799999999999, 5.793377999999998, 23.17351199999999, 1],
 ['1999-11-17', 90.25, 6.276159499999998, 25.10463799999999, 1],
 ['1999-11-18', 89.625, 6.758940999999997, 27.03576399999999, 1],
 ['1999-11-19', 92.43799999999999, 7.241722499999997, 28.96688999999999, 1],
 ['1999-11-22', 90.625

In [ ]:
AAPL_Location = r'C:\Users\Eazy\Documents\CSE\CSE410\Project\PE_EPS_ROE\AAPL_quarterly_financial_data.csv'
aapl = pd.read_csv(AAPL_Location)
aapl_qtly_end_date = aapl['Quarter end'].to_numpy()
aapl_qtly_shares = aapl['Shares'].to_numpy()
aapl_qtly_shares_adj = aapl['Shares split adjusted'].to_numpy()
aapl_qtly_shares_splitFac = aapl['Split factor'].to_numpy()
aapl_qtly_shares_asset = aapl['Assets'].to_numpy()
aapl_qtly_shares_liability = aapl['Liabilities'].to_numpy()
aapl_qtly_shares_equity = aapl['Shareholders equity'].to_numpy()

In [ ]:
#result = np.where(aapl_qtly_end_date == "6/290/2019")
#result

In [ ]:
aapl_vec = [];
aapl_dates = [];
for i in range(len(aapl_qtly_end_date)-1):
    date2 = aapl_qtly_end_date[i]
    date22 = date2.split("/");
    date1 = aapl_qtly_end_date[i+1]
    date12 = date1.split("/");
    start_date = date(int(date12[2]), int(date12[0]),int(date12[1]))
    end_date = date(int(date22[2]), int(date22[0]),int(date22[1]))
    delta = timedelta(days=1)
    start_date += delta
    end_date += delta
    
    temp_dates = [];
    
    start_shares = aapl_qtly_shares_adj[np.where(aapl_qtly_end_date == date1)];
    end_shares = aapl_qtly_shares_adj[np.where(aapl_qtly_end_date == date2)];
    
    start_spfac = aapl_qtly_shares_splitFac[np.where(aapl_qtly_end_date == date1)];
    end_spfac = aapl_qtly_shares_splitFac[np.where(aapl_qtly_end_date == date2)];
    
    start_asset = aapl_qtly_shares_asset[np.where(aapl_qtly_end_date == date1)];
    end_asset = aapl_qtly_shares_asset[np.where(aapl_qtly_end_date == date2)];
    
    start_liability = aapl_qtly_shares_liability[np.where(aapl_qtly_end_date == date1)];
    end_liability = aapl_qtly_shares_liability[np.where(aapl_qtly_end_date == date2)];
    
    start_equity = aapl_qtly_shares_equity[np.where(aapl_qtly_end_date == date1)];
    end_equity = aapl_qtly_shares_equity[np.where(aapl_qtly_end_date == date2)];
    
    while start_date < end_date-delta:
        datestr = start_date.strftime("%Y-%m-%d")
        start_date += delta
        temp_dates.insert(0,datestr)
        #print(start_date)
    temp_dates.insert(0,(end_date-delta).strftime("%Y-%m-%d"))
    
    delta_shares = (end_shares-start_shares)/len(temp_dates);
    delta_spfac = (end_spfac-start_spfac)/len(temp_dates);
    delta_asset = (end_asset-start_asset)/len(temp_dates);
    delta_liability = (end_liability-start_liability)/len(temp_dates);
    delta_equity = (end_equity-start_equity)/len(temp_dates);
    
    temp_vec = [];
    
    for j in range(len(temp_dates)-1):
        temp_vec.insert(0,(start_shares+j*delta_shares,start_spfac+j*delta_spfac,start_asset+j*delta_asset,start_liability+j*delta_liability,start_equity+j*delta_equity))
    #temp_vec.insert(0,(end_shares-delta_shares))
    temp_vec.insert(0,(end_shares-delta_shares,end_spfac-delta_spfac,end_asset-delta_asset,end_liability-delta_liability,end_equity-delta_equity))
    
    aapl_vec += temp_vec;
    aapl_dates += temp_dates;
    

In [ ]:
aapl_vec